In [ ]:
"""
Code to fine-tune a bert model on hatexplain dataset
I/p: Sentence, Explanation, Keywords (Only from Hugchat)
O/p: Hate-Speech Classification
"""


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install transformers[torch]
!pip install datasets
!pip install evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 83.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 34.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 57.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 89.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.6/227.6 kB 35.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 486.2/486.2 kB 25.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 31.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 21.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 9.6 MB/s eta 0:00:00


In [3]:
from datasets import load_dataset
import ast
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from transformers import TrainingArguments
from transformers import Trainer
from transformers import EarlyStoppingCallback, IntervalStrategy
import numpy as np
import evaluate
import torch
import os
import pandas as pd
os.environ["WANDB_DISABLED"] = "true"

In [4]:
### Input
train_path = '/content/drive/MyDrive/cs4nlp/CS4NLP-HateXplain/data/bert_modeling/df_train.csv'
val_path = '/content/drive/MyDrive/cs4nlp/CS4NLP-HateXplain/data/bert_modeling/df_val.csv'
test_path = '/content/drive/MyDrive/cs4nlp/CS4NLP-HateXplain/data/bert_modeling/df_test.csv'

### Output
train_processed_path = '/content/drive/MyDrive/cs4nlp/CS4NLP-HateXplain/data/bert_modeling/df_train_processed.csv'
val_processed_path = '/content/drive/MyDrive/cs4nlp/CS4NLP-HateXplain/data/bert_modeling/df_val_processed.csv'
test_processed_path = '/content/drive/MyDrive/cs4nlp/CS4NLP-HateXplain/data/bert_modeling/df_test_processed.csv'
model_path = '/content/drive/MyDrive/cs4nlp/CS4NLP-HateXplain/data/bert_modeling/bert_model_v2.pt'

In [5]:
def load_data(path):
    df = pd.read_csv(path)
    return df

def save_data(df, path):
    df.to_csv(path, index=False)

In [7]:
df_train = load_data(train_path)
df_train.head()

,id,unmasked_sentence,hugchat_keywords_processed,hugchat_explanation,mix_keywords,gt_comm_target,gt_label
0,23107796_gab,u really think i would not have been raped by ...,"['muslim', 'nazi', 'neo', 'raped', 'rape', 'fe...",the language used in the sentence suggests an...,"['raped', 'feral', 'hindu', 'muslim', 'neo', '...","['Islam', 'Other', 'Hindu']",offensive
1,9995600_gab,the uk has threatened to return radioactive wa...,"['migrants', 'waste', 'radioactive', 'paki']",this statement contains derogatory language t...,"['send', 'back', 'all', 'the', 'paki', 'migran...","['Islam', 'Indian', 'Minority', 'Refugee']",offensive
2,1227920812235051008_twitter,if english is not imposition then hindi is als...,"['imposition', 'hindi', 'shut', 'retards', 'ch...",this sentence contains several derogatory ter...,"['chutiya', 'retards', 'stophindiimposition']","['Other', 'Hindu']",offensive
3,1204931715778543624_twitter,no liberal congratulated hindu refugees post c...,"['refugees', 'cab', 'hindus', 'liberal', 'hind...",the statement seems factually correct as ther...,"['hate', 'hindus']",['Hindu'],offensive
4,1179102559241244672_twitter,he said bro even your texts sound redneck what...,"['redneck', 'bro']",this sentence appears to be casual conversati...,['redneck'],"['Caucasian', 'Economic']",offensive


In [8]:
def preprocess(in_path, out_path):
    df = load_data(in_path)
    df_copy = df.copy()
    df_copy['hugchat_keywords_processed'] = df_copy['hugchat_keywords_processed'].apply(str).apply(lambda x: ' '.join(k for k in ast.literal_eval(x)))
    df_copy['gt_comm_target'] = df_copy['gt_comm_target'].apply(lambda x: ' '.join(k for k in x))
    df_copy['bert_ip1'] = df_copy.apply(lambda row: row['unmasked_sentence'] + '.' + row['hugchat_explanation'], axis=1)
    keywords_prefix = " keywords suggesting this explanation are: "
    df_copy['bert_ip2'] = df_copy.apply(lambda row: row['unmasked_sentence'] + '.' + row['hugchat_explanation'] + keywords_prefix + row['hugchat_keywords_processed'], axis=1)
    df_copy['gt_label'] = df_copy['gt_label'].map({'hate':0, 'offensive':2, 'normal':1})
    df_copy = df_copy[['bert_ip1','bert_ip2','gt_label']].copy()
    df_copy.rename(columns={'gt_label':'label'}, inplace=True)
    save_data(df_copy, out_path)
    return df_copy


In [9]:
def tokenize_data(example):
    return tokenizer(example['bert_ip2'], padding='max_length')

In [10]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)


In [11]:
df_train = preprocess(train_path, train_processed_path)
df_val = preprocess(val_path, val_processed_path)
df_test = preprocess(test_path, test_processed_path)

In [12]:
dataset = load_dataset('csv', data_files={'train': train_processed_path,
                                          'val' : val_processed_path,
                                          'test': test_processed_path}, encoding = "ISO-8859-1")

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating val split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-396e34e2d57832fe/0.0.0/eea64c71ca8b46dd3f537ed218fc9bf495d5707789152eb2764f5c78fa66d59d. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [13]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')
dataset = dataset.map(tokenize_data, batched=True)
model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=3)

Map:   0%|          | 0/14057 [00:00<?, ? examples/s]

Map:   0%|          | 0/1786 [00:00<?, ? examples/s]

Map:   0%|          | 0/1759 [00:00<?, ? examples/s]

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initi

In [14]:
metric = evaluate.load("accuracy")
training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/cs4nlp/CS4NLP-HateXplain/data/bert_modeling/",
    num_train_epochs=10,
    learning_rate=2e-5,
    evaluation_strategy = IntervalStrategy.STEPS, # "steps"
    eval_steps = 50, # Evaluation and Save happens every 50 steps
    save_total_limit = 5, # Only last 5 models are saved. Older ones are deleted
    weight_decay=0.01,
    load_best_model_at_end=True)

train_dataset = dataset['train']
val_dataset = dataset['val']

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset = train_dataset,
    eval_dataset = val_dataset,
    compute_metrics = compute_metrics,
    callbacks = [EarlyStoppingCallback(early_stopping_patience=10)]
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [15]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss,Validation Loss,Accuracy
50,No log,1.069386,0.404255
100,No log,0.965495,0.565510
150,No log,0.853003,0.601904
200,No log,0.828944,0.633259
250,No log,0.830554,0.630459
300,No log,0.814159,0.636618
350,No log,0.854099,0.625980
400,No log,0.810509,0.644457
450,No log,0.826651,0.638858
500,0.898000,0.792646,0.657335


TrainOutput(global_step=1500, training_loss=0.8111163330078125, metrics={'train_runtime': 813.8513, 'train_samples_per_second': 172.722, 'train_steps_per_second': 21.601, 'total_flos': 3157361012736000.0, 'train_loss': 0.8111163330078125, 'epoch': 0.85})

In [16]:
model_path = '/content/drive/MyDrive/cs4nlp/CS4NLP-HateXplain/data/bert_modeling/bert_model_v2.pt'
torch.save(model.state_dict(),model_path)
import pickle
test_dataset = dataset['test']
predictions = trainer.predict(test_dataset)
with open('/content/drive/MyDrive/cs4nlp/CS4NLP-HateXplain/data/bert_modeling/predictions_try2.pkl', 'wb') as handle:
  pickle.dump(predictions, handle)
print(predictions)

PredictionOutput(predictions=array([[-1.3491249 ,  1.1257336 ,  0.3904664 ],
       [-1.4945644 , -0.6119802 ,  1.7896495 ],
       [ 1.8660423 , -1.2407206 , -0.8424918 ],
       ...,
       [-1.4224671 , -0.6037438 ,  1.71408   ],
       [ 2.709871  , -1.6669953 , -0.41652632],
       [-0.9940004 ,  1.2881019 ,  0.25906497]], dtype=float32), label_ids=array([1, 2, 0, ..., 2, 0, 0]), metrics={'test_loss': 0.7566914558410645, 'test_accuracy': 0.6719727117680501, 'test_runtime': 14.8681, 'test_samples_per_second': 118.307, 'test_steps_per_second': 14.797})


In [17]:
predictions[2]

{'test_loss': 0.7566914558410645,
 'test_accuracy': 0.6719727117680501,
 'test_runtime': 14.8681,
 'test_samples_per_second': 118.307,
 'test_steps_per_second': 14.797}